In [1]:
import pandas as pd
import sys
import os
from os import listdir
os.chdir("c:/Users/Robert/Documents/Projekte/dev/sport_betting/")
import config as CONFIG
pd.set_option('display.max_columns', 500)

In [2]:
from src.data.provide_data import get_model_data
from src.models.evaluate import custom_classification_report, custom_lazy_report



In [11]:
X_train, y_train = get_model_data(filename = "Train")
X_valid, y_valid = get_model_data(filename = "Valid")
X_test, y_test = get_model_data(filename = "Test")

cat = [X_train.columns.get_loc(i) for i in ["Team","Div","Opponent"]]

Train 5246
Valid 1952
Test 1952


In [12]:
features = X_train.columns

In [13]:
from sklearn.preprocessing import StandardScaler

# Initialize scaler object
scaler = StandardScaler()

# Fit scaler on training data
scaler.fit(X_train)

# Scale training and test data
X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

In [8]:
import pickle
clf = pickle.load(open(CONFIG.DATA_FOLDER_MODELS+'lgb.pkl', 'rb'))

c:\Users\Robert\Documents\Projekte\dev\sport_betting\.venv\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Baseline Model

In [ ]:
pickle.loads(clf, open(CONFIG.DATA_FOLDER_MODELS + 'lgb.pkl', 'wb'))

In [9]:
from scripts.02_process_data.preprocess_team_opponent import preprocess_data

SyntaxError: invalid decimal literal (1773067971.py, line 1)

## Gridsearch für bestes Baseline Model

In [36]:
fi = pd.DataFrame() 
fi['features'] = features
fi['importance'] = clf.booster_.feature_importance(importance_type='gain')  
fi['importance'] = fi['importance']/sum(fi['importance'])
fi.sort_values(by='importance',ascending=False).head(10)

,features,importance
7,B365_Team,0.468098
30,B365_Team_odd_pred,0.142169
36,BW_Team_odd_pred,0.077514
22,Avg_Opponent,0.047664
17,Avg_Team,0.039862
3,BW_Draw,0.036606
2,BW_Opponent,0.031168
26,Min_Draw,0.028843
15,Max_Team,0.025619
5,B365_Opponent,0.025413


,features,importance
30,B365_Team_odd_pred,0.138571
7,B365_Team,0.127143
2,BW_Opponent,0.078571
3,BW_Draw,0.058571
15,Max_Team,0.055714
8,IW_Opponent,0.052857
22,Avg_Opponent,0.048571
29,Ratio_Draw,0.047143
44,Span_Draw_last_4_games,0.047143
39,B365_Team_last_4_games,0.044286


# Transformer

## Pretrainer

## Modellfitting

In [ ]:
pickle.load(open('tabnet.pkl', 'rb'))

## Evaluierung

In [ ]:
from matplotlib import pyplot as plt

plt.plot(clf.history['loss'], label='Validation Loss')
plt.plot(clf.history['valid_auc'], label='AUC')
plt.plot(clf.history['valid_balanced_accuracy'], label='Accuracy')

plt.legend()
plt.show()

from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
r = classification_report(y_pred=y_pred,y_true=y_test)
print(r)

## Erklärbarkeit

In [ ]:
f_i = pd.Series(clf.feature_importances_)
f_i.index = features
f_i.sort_values(ascending=False)

In [ ]:
"BW_opponent_odd_pred","B365_Opponent","Avg_Opponent","IW_Team","Span_Draw"      

In [ ]:
sum(f_i>0.01)

#  Hypertune

In [ ]:
import optuna
from sklearn.neighbors import NearestCentroid
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, brier_score_loss

def load_dataset():
    # Load your dataset here
    # Replace this with your dataset loading code
    # X and y should be your feature and target variables
    pass

def objective(trial):
    # Define the hyperparameters to optimize
    shrink_threshold = trial.suggest_float("shrink_threshold", 0.0, 1.0)
    
    X_train, y_train = get_model_data(filename = "Train")
    X_valid, y_valid = get_model_data(filename = "Valid")
    X_test, y_test = get_model_data(filename = "Test")
    
    # Create and train the NearestCentroid classifier with the suggested parameters
    clf = NearestCentroid(shrink_threshold=shrink_threshold)
    clf.fit(X_train, y_train)
    
    # Make predictions on the validation set
    y_pred = clf.predict(X_valid)
    
    # Calculate the accuracy of the model
    score = brier_score_loss(y_valid, y_pred)
    
    return brier_score_loss

if __name__ == "__main__":
    # Load your dataset
    
    # Specify the SQLite database file for the study
    study_name = "neareast_centroid_optimization.db"
    study = optuna.create_study(direction="minimize", study_name=study_name)

    # Optimize the study
    study.optimize(objective, n_trials=100)  # You can adjust the number of trials

    # Get the best parameters and their corresponding accuracy
    best_params = study.best_params
    best_accuracy = study.best_value

    print("Best Parameters:", best_params)
    print("Best Accuracy:", best_accuracy)

    # # Save the study to the specified SQLite database file
    # study.trials_dataframe().to_sql(study_name, "sqlite:///{}".format(study_name), if_exists="replace")


In [ ]:
    clf = NearestCentroid(shrink_threshold=0.95)
    clf.fit(X_train, y_train)

In [ ]:
clf.predict(X_test)